# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U -q PyDrive
!pip install httplib2==0.15.0
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFileList
from google.colab import auth
from oauth2client.client import GoogleCredentials

from getpass import getpass
import urllib

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Cloning PAL_2021 to access modules.
# Need password to access private repo.

if 'CLIPPER' not in os.listdir():
    cmd_string = 'git clone https://github.com/PAL-ML/CLIPPER.git'
    os.system(cmd_string)

# Installation

## Install multi label metrics dependencies

In [ ]:
! pip install scikit-learn==0.24

## Install CLIP dependencies

In [ ]:
# import subprocess

# CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
# print("CUDA version:", CUDA_version)

# if CUDA_version == "10.0":
#     torch_version_suffix = "+cu100"
# elif CUDA_version == "10.1":
#     torch_version_suffix = "+cu101"
# elif CUDA_version == "10.2":
#     torch_version_suffix = ""
# else:
#     torch_version_suffix = "+cu110"

In [ ]:
# ! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

In [ ]:
# ! pip install ftfy regex
# ! wget https://openaipublic.azureedge.net/clip/bpe_simple_vocab_16e6.txt.gz -O bpe_simple_vocab_16e6.txt.gz

In [ ]:
# !pip install git+https://github.com/Sri-vatsa/CLIP # using this fork because of visualization capabilities

## Install clustering dependencies

In [ ]:
!pip -q install umap-learn>=0.3.7

## Install dataset manager dependencies

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=b82e02abf79b34dcd61f014b36e9c2859d45262a74995db4d82a855fb53cfb38
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


# Imports

In [ ]:
# ML Libraries
import tensorflow as tf
import tensorflow_hub as hub
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import keras

# Data processing
import PIL
import base64
import imageio
import pandas as pd
import numpy as np
import json

from PIL import Image
import cv2
from sklearn.feature_extraction.image import extract_patches_2d

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from IPython.core.display import display, HTML
from matplotlib import cm
import matplotlib.image as mpimg

# Models
# import clip

# Datasets
import tensorflow_datasets as tfds

# Clustering
# import umap

from sklearn import metrics
from sklearn.cluster import KMeans
#from yellowbrick.cluster import KElbowVisualizer

# Misc
import progressbar
import logging
from abc import ABC, abstractmethod
import time
import urllib.request
import os
from sklearn.metrics import jaccard_score, hamming_loss, accuracy_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer


# Modules
# from CLIPPER.code.ExperimentModules import embedding_models
from CLIPPER.code.ExperimentModules.dataset_manager import DatasetManager
from CLIPPER.code.ExperimentModules.weight_imprinting_classifier import WeightImprintingClassifier
from CLIPPER.code.ExperimentModules import simclr_data_augmentations
from CLIPPER.code.ExperimentModules.utils import (save_npy, load_npy, 
                                                       get_folder_id, 
                                                       create_expt_dir, 
                                                       save_to_drive, 
                                                       load_all_from_drive_folder, 
                                                       download_file_by_name, 
                                                       delete_file_by_name)

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


# Initialization & Constants

## Dataset details

In [ ]:
folder_name = "ImagenetR-Embeddings-28-02-21"

# Change parentid to match that of experiments root folder in gdrive
parentid = '1bK72W-Um20EQDEyChNhNJthUNbmoSEjD'

# Filepaths
test_labels_filename = "test_labels.npz"

test_embeddings_filename_suffix = "_embeddings_test.npz"

# Initialize sepcific experiment folder in drive
folderid = create_expt_dir(drive, parentid, folder_name)

title: ImagenetR-Embeddings-28-02-21, id: 1rACsi_OuN_WITrsuZt7pGZYUrUA2u6xT
Experiment folder already exists. WARNING: Following with this run might overwrite existing results stored.


## Few shot learning parameters

In [ ]:
num_ways = 5 # [5, 20]
num_shot = 5 # [5, 1]
num_eval = 15 # [5, 10, 15, 19]
num_episodes = 100
shuffle = False

# Load data

In [ ]:
def get_ndarray_from_drive(drive, folderid, filename):
    download_file_by_name(drive, folderid, filename)
    return np.load(filename)['data']

test_labels = get_ndarray_from_drive(drive, folderid, test_labels_filename)

# Create label dictionary

In [ ]:
unique_labels = np.unique(test_labels)
print(len(unique_labels))

200


In [ ]:
label_dictionary = {la:[] for la in unique_labels}

for i in range(len(test_labels)):
    la = test_labels[i]

    label_dictionary[la].append(i)

# Weight Imprinting models on train data embeddings

## Function definitions

In [ ]:
def calculate_single_label_accuracy(pred, y, label_mapping):
    x = 0
    for i, p in enumerate(pred):
        pred_label = label_mapping[p]
        if pred_label == y[i]:
            x += 1
    x = x/(i+1)

    return x

In [ ]:
def start_progress_bar(bar_len):
    widgets = [
        ' [', 
        progressbar.Timer(format= 'elapsed time: %(elapsed)s'), 
        '] ', 
        progressbar.Bar('*'),' (', 
        progressbar.ETA(), ') ', 
        ]
    pbar = progressbar.ProgressBar(
        max_value=bar_len, widgets=widgets
        ).start()
    return pbar

In [ ]:
def run_evaluations(
    embeddings, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
    verbose=True,
    metrics=["accuracy", "c_f1"]
):
    accuracies = []
    f1_scores = []

    if verbose:
        pbar = start_progress_bar(num_episodes)

    for i in range(num_episodes):
        wi_x = embeddings[train_indices[i]]
        wi_x = WeightImprintingClassifier.preprocess_input(wi_x)
        eval_x = embeddings[eval_indices[i]]
        eval_x = WeightImprintingClassifier.preprocess_input(eval_x)

        wi_weights, label_mapping = WeightImprintingClassifier.get_imprinting_weights(
            wi_x, wi_y[i], False
            )

        wi_parameters = {
            "num_classes": num_ways,
            "input_dims": wi_x.shape[-1],
            "scale": False,
            "dense_layer_weights": wi_weights
        }

        wi_cls = WeightImprintingClassifier(wi_parameters)

        # Evaluate the weight imprinting model
        metric_vals = wi_cls.evaluate_single_label_metrics(eval_x, eval_y[i], label_mapping, metrics=metrics)
        if "accuracy" in metrics:
            accuracies.append(metric_vals["accuracy"])  
        if "c_f1" in metrics:
            f1_scores.append(metric_vals["c_f1"])

        del wi_x
        del eval_x
        del wi_cls

        if verbose:
            pbar.update(i+1)

    metric_arrays = []
    if "accuracy" in metrics:
        metric_arrays.append(accuracies)
    if "c_f1" in metrics:
        metric_arrays.append(f1_scores)

    return metric_arrays

## Picking indices

In [ ]:
eval_indices = []
train_indices = []
wi_y = []
eval_y = []

label_dictionary = {la:label_dictionary[la] for la in label_dictionary if len(label_dictionary[la]) >= (num_shot+num_eval)}
unique_labels = list(label_dictionary.keys())

pbar = start_progress_bar(num_episodes)

for s in range(num_episodes):
    # Setting random seed for replicability
    np.random.seed(s)

    _train_indices = []
    _eval_indices = []

    selected_labels = np.random.choice(unique_labels, size=num_ways, replace=False)
    for la in selected_labels:
        la_indices = label_dictionary[la]
        select = np.random.choice(la_indices, size = num_shot+num_eval, replace=False)
        tr_idx = list(select[:num_shot])
        ev_idx = list(select[num_shot:])

        _train_indices = _train_indices + tr_idx
        _eval_indices = _eval_indices + ev_idx

    if shuffle:
        np.random.shuffle(_train_indices)
        np.random.shuffle(_eval_indices)

    train_indices.append(_train_indices)
    eval_indices.append(_eval_indices)

    _wi_y = test_labels[_train_indices]
    _eval_y = test_labels[_eval_indices]

    wi_y.append(_wi_y)
    eval_y.append(_eval_y)

    pbar.update(s+1)

 [elapsed time: 0:00:00] |**************************        | (ETA:   0:00:00) 

## Inception V3

In [ ]:
# Load numpy data from drive
inceptionv3_embeddings_test_fn = "inceptionv3" + test_embeddings_filename_suffix

inceptionv3_embeddings_test = get_ndarray_from_drive(drive, folderid, inceptionv3_embeddings_test_fn)

In [ ]:
inceptionv3_accuracies, inceptionv3_f1_scores = run_evaluations(
    inceptionv3_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:00] |                                  | (ETA:  --:--:--) 

 [elapsed time: 0:00:08] |********************************* | (ETA:   0:00:00) 

In [ ]:
inceptionv3_mean_accuracy = np.mean(inceptionv3_accuracies)
print("Inceptionv3 Mean accuracy: ", inceptionv3_mean_accuracy)
inceptionv3_mean_f1_score = np.mean(inceptionv3_f1_scores)
print("Inceptionv3 Mean f1 score: ", inceptionv3_mean_f1_score)

Inceptionv3 Mean accuracy:  0.17725
Inceptionv3 Mean f1 score:  0.16886249089512234


## Resnet 50

In [ ]:
resnet50_embeddings_test_fn = "resnet50" + test_embeddings_filename_suffix
resnet50_embeddings_test = get_ndarray_from_drive(drive, folderid, resnet50_embeddings_test_fn)


In [ ]:
resnet50_accuracies, resnet50_f1_scores = run_evaluations(
    resnet50_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:08] |********************************* | (ETA:   0:00:00) 

In [ ]:
resnet50_mean_accuracy = np.mean(resnet50_accuracies)
print("Resnet 50 Mean accuracy: ", resnet50_mean_accuracy)
resnet50_mean_f1_score = np.mean(resnet50_f1_scores)
print("Resnet 50 Mean f1 score: ", resnet50_mean_f1_score)

Resnet 50 Mean accuracy:  0.24460000000000004
Resnet 50 Mean f1 score:  0.2342756323393944


## MoCo Resnet

In [ ]:
moco_resnet50_embeddings_test_fn = "moco_resnet50" + test_embeddings_filename_suffix

moco_resnet50_embeddings_test = get_ndarray_from_drive(drive, folderid, moco_resnet50_embeddings_test_fn)

In [ ]:
moco_resnet50_accuracies, moco_resnet50_f1_scores = run_evaluations(
    moco_resnet50_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:09] |********************************* | (ETA:   0:00:00) 

In [ ]:
moco_resnet50_mean_accuracy = np.mean(moco_resnet50_accuracies)
print("Moco Resnet Mean accuracy: ", moco_resnet50_mean_accuracy)
moco_resnet50_mean_f1_score = np.mean(moco_resnet50_f1_scores)
print("Moco Resnet Mean f1 score: ", moco_resnet50_mean_f1_score)

Moco Resnet Mean accuracy:  0.19730000000000003
Moco Resnet Mean f1 score:  0.1931982487698186


## PCL Resnet

In [ ]:
pcl_resnet50_embeddings_test_fn = "pcl_resnet50" + test_embeddings_filename_suffix

pcl_resnet50_embeddings_test = get_ndarray_from_drive(drive, folderid, pcl_resnet50_embeddings_test_fn)

In [ ]:
pcl_resnet50_accuracies, pcl_resnet50_f1_scores = run_evaluations(
    pcl_resnet50_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:08] |********************************* | (ETA:   0:00:00) 

In [ ]:
pcl_resnet50_mean_accuracy = np.mean(pcl_resnet50_accuracies)
print("PCL Resnet Mean accuracy: ", pcl_resnet50_mean_accuracy)
pcl_resnet50_mean_f1_score = np.mean(pcl_resnet50_f1_scores)
print("PCL Resnet Mean f1 score: ", pcl_resnet50_mean_f1_score)

PCL Resnet Mean accuracy:  0.21015
PCL Resnet Mean f1 score:  0.20530956189416


## SwAV Resnet

In [ ]:
swav_resnet50_embeddings_test_fn = "swav_resnet50" + test_embeddings_filename_suffix

swav_resnet50_embeddings_test = get_ndarray_from_drive(drive, folderid, swav_resnet50_embeddings_test_fn)

In [ ]:
swav_resnet50_accuracies, swav_resnet50_f1_scores = run_evaluations(
    swav_resnet50_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:08] |********************************* | (ETA:   0:00:00) 

In [ ]:
swav_resnet50_mean_accuracy = np.mean(swav_resnet50_accuracies)
print("Swav Resnet Mean accuracy: ", swav_resnet50_mean_accuracy)
swav_resnet50_mean_f1_score = np.mean(swav_resnet50_f1_scores)
print("Swav Resnet Mean f1 score: ", swav_resnet50_mean_f1_score)

Swav Resnet Mean accuracy:  0.14989999999999998
Swav Resnet Mean f1 score:  0.1386389817393805


## SimCLR

In [ ]:
simclr_embeddings_test_fn = "simclr" + test_embeddings_filename_suffix

simclr_embeddings_test = get_ndarray_from_drive(drive, folderid, simclr_embeddings_test_fn)

In [ ]:
simclr_accuracies, simclr_f1_scores = run_evaluations(
    simclr_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:08] |********************************* | (ETA:   0:00:00) 

In [ ]:
simclr_mean_accuracy = np.mean(simclr_accuracies)
print("Simclr Mean accuracy: ", simclr_mean_accuracy)
simclr_mean_f1_score = np.mean(simclr_f1_scores)
print("Simclr Mean f1 score: ", simclr_mean_f1_score)

Simclr Mean accuracy:  0.2138
Simclr Mean f1 score:  0.20967300000205447


## VGG16

In [ ]:
vgg16_embeddings_test_fn = "vgg16" + test_embeddings_filename_suffix

vgg16_embeddings_test = get_ndarray_from_drive(drive, folderid, vgg16_embeddings_test_fn)

In [ ]:
vgg16_accuracies, vgg16_f1_scores = run_evaluations(
    vgg16_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:08] |********************************* | (ETA:   0:00:00) 

In [ ]:
vgg16_mean_accuracy = np.mean(vgg16_accuracies)
print("VGG16 Mean accuracy: ", vgg16_mean_accuracy)
vgg16_mean_f1_score = np.mean(vgg16_f1_scores)
print("VGG16 Mean f1 score: ", vgg16_mean_f1_score)

VGG16 Mean accuracy:  0.21305000000000004
VGG16 Mean f1 score:  0.21215224645054934


## CLIP

In [ ]:
clip_embeddings_test_fn = "clip" + test_embeddings_filename_suffix

clip_embeddings_test = get_ndarray_from_drive(drive, folderid, clip_embeddings_test_fn)

In [ ]:
clip_accuracies, clip_f1_scores = run_evaluations(
    clip_embeddings_test, 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
)

 [elapsed time: 0:00:08] |********************************* | (ETA:   0:00:00) 

In [ ]:
clip_mean_accuracy = np.mean(clip_accuracies)
print("CLIP Mean accuracy: ", clip_mean_accuracy)
clip_mean_f1_score = np.mean(clip_f1_scores)
print("CLIP Mean f1 score: ", clip_mean_f1_score)

CLIP Mean accuracy:  0.305
CLIP Mean f1 score:  0.2841753094222577


# Conclusion

In [ ]:
mean_accuracy_dict={
    inceptionv3_mean_accuracy: "Inception V3",
    resnet50_mean_accuracy: "Resnet 50",
    moco_resnet50_mean_accuracy: "MoCo Resnet 50",
    pcl_resnet50_mean_accuracy: "PCL Resnet 50",
    swav_resnet50_mean_accuracy: "SwAV Resnet 50",
    simclr_mean_accuracy: "SimCLR",
    vgg16_mean_accuracy: "VGG 16", 
    clip_mean_accuracy: "CLIP"
}

acc_vals = sorted(list(mean_accuracy_dict.keys()), reverse=True)

In [ ]:
mean_f1_score_dict={
    inceptionv3_mean_f1_score: "Inception V3",
    resnet50_mean_f1_score: "Resnet 50",
    moco_resnet50_mean_f1_score: "MoCo Resnet 50",
    pcl_resnet50_mean_f1_score: "PCL Resnet 50",
    swav_resnet50_mean_f1_score: "SwAV Resnet 50",
    simclr_mean_f1_score: "SimCLR",
    vgg16_mean_f1_score: "VGG 16", 
    clip_mean_f1_score: "CLIP"
}

f1_vals = sorted(list(mean_f1_score_dict.keys()), reverse=True)

In [ ]:
print("All accuracies for {} way {} shot classification on ImagenetR with {} random episodes- ".format(
    num_ways, num_shot, num_episodes
    ))
for val in acc_vals:
    print(mean_accuracy_dict[val], ": ", val)

All accuracies for 20 way 1 shot classification on ImagenetR with 100 random episodes- 
CLIP :  0.305
Resnet 50 :  0.24460000000000004
SimCLR :  0.2138
VGG 16 :  0.21305000000000004
PCL Resnet 50 :  0.21015
MoCo Resnet 50 :  0.19730000000000003
Inception V3 :  0.17725
SwAV Resnet 50 :  0.14989999999999998


In [ ]:
print("All f1 scores for {} way {} shot classification on Omniglot with {} random episodes- ".format(
    num_ways, num_shot, num_episodes
    ))
for val in f1_vals:
    print(mean_f1_score_dict[val], ": ", val)

All f1 scores for 20 way 1 shot classification on Omniglot with 100 random episodes- 
CLIP :  0.2841753094222577
Resnet 50 :  0.2342756323393944
VGG 16 :  0.21215224645054934
SimCLR :  0.20967300000205447
PCL Resnet 50 :  0.20530956189416
MoCo Resnet 50 :  0.1931982487698186
Inception V3 :  0.16886249089512234
SwAV Resnet 50 :  0.1386389817393805
